In [4]:
# Carregando dados
from pathlib import Path
import fitz
import re 
import pandas as pd 

ModuleNotFoundError: No module named 'fitz'

In [ ]:
def get_bimestre_from_date(date_str):
    # date_str no formato '01.04' ou '01 ABR' etc
    # Aqui você transforma em mês e decide o bimestre (1 a 4)
    import datetime
    try:
        # Exemplo com '01.04' (dia.mês)
        day, month = map(int, date_str.split('.')[:2])
    except:
        # Pode adaptar para o formato '01 ABR' convertendo ABR para 4
        month_str_map = {
            "JAN": 1, "FEV": 2, "MAR":3, "ABR":4,
            "MAI":5, "JUN":6, "JUL":7, "AGO":8,
            "SET":9, "OUT":10, "NOV":11, "DEZ":12
        }
        parts = date_str.split()
        day = int(parts[0])
        month = month_str_map.get(parts[1].upper(), 1)
    
    if month in [1,2,3]:
        return 1
    elif month in [4,5,6]:
        return 2
    elif month in [7,8,9]:
        return 3
    else:
        return 4


In [ ]:
def get_ano_from_path(path):
    # Exemplo: path.parts pode ter "matematica 2024"
    for part in path.parts:
        match = re.search(r"(\d{4})", part)
        if match:
            return int(match.group(1))
    return None

In [ ]:
def extrat_text_pdfs_in_folder(folder_path):
    """Extrai textos de todos PDFs dentro de uma pasta (não recursivo)"""
    folder = Path(folder_path)
    texts = []
    pdf_files = list(folder.glob("*.pdf"))
    for pdf_file in pdf_files:
        print(f"Lendo arquivo: {pdf_file.name}")
        doc = fitz.open(str(pdf_file))
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        texts.append((pdf_file, text))
    return texts

In [ ]:
def extract_competencias(text):
    # tenta encontrar a seção "Competências" ou "Habilidades"
    match = re.search(r"(Competências|Habilidades| Expectativas de Aprendizagem)\s*:(.*?)(\n\n|$)", text, re.DOTALL)
    if match:
        competencias_text = match.group(2).strip()
        # separar por linha ou por '-', como for o padrão
        competencias_list = re.findall(r"[-•]\s*(.*)", competencias_text)
        if competencias_list:
            return competencias_list
        else:
            return [competencias_text]  # pode ser só uma frase
    return []

In [ ]:
path = Path().resolve() #/ "bases/matematica 2024/01.04 a 14.04.2024_ 1A_ Planejamento de Matemática"
current_file = path / "bases/conteudo"



In [ ]:
def extrat_text_pasta(folder_path):
    folder = Path(folder_path)
    all_text = ""
    pdf_files = list(folder.glob("*.pdf"))  # pega todos PDFs na pasta

    for pdf_file in pdf_files:
        print(f"Lendo arquivo: {pdf_file.name}")
        doc = fitz.open(str(pdf_file))
        for page in doc:
            all_text += page.get_text()
        doc.close()

    return all_text

In [ ]:
def clean_text(text):
    text = re.sub(r"http://professor:80.*?\n", "", text)
    text = re.sub(r"Emitido em.*?\n", "", text)
    text = re.sub(r"Pagina: \d+ de \d+", "", text)
    text = re.sub(r"[_]{3,}", "", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text 

In [ ]:
def split_by_lessons(text, path, materia):
    meses = "JAN|FEV|MAR|ABR|MAI|JUN|JUL|AGO|SET|OUT|NOV|DEZ"
    blocks = re.split(rf"(?=\d{{1,2}} ({meses}))", text, flags=re.IGNORECASE)
    lessons = []
    ano = get_ano_from_path(path)
    for block in blocks:
        date_match = re.match(rf"(\d{{1,2}} ({meses}))", block, flags=re.IGNORECASE)
        if date_match:
            date = date_match.group(1)
            bimestre = get_bimestre_from_date(date)
            lesson_data = {
                "ano": ano,
                "bimestre": bimestre,
                "materia": materia,
                "data": date,
                "texto": block.strip(),
                "competencias": extract_competencias(block)
            }
            lessons.append(lesson_data)
    return lessons


In [ ]:
def export_to_csv(lessons, output_csv="aulas_bimestre1.csv"):
    df = pd.DataFrame(lessons)
    df.to_csv(output_csv, index=False)
    print(f"[X] Exportando para {output_csv}")

In [ ]:
todas_lessons = []

for materia_folder in current_file.iterdir():
    if materia_folder.is_dir():
        materia = materia_folder.name.lower()
        print(f"\nProcessando matéria: {materia}")
        textos = extrat_text_pdfs_in_folder(materia_folder)
        for (pdf_file, texto) in textos:
            texto_limpo = clean_text(texto)
            lessons = split_by_lessons(texto_limpo, pdf_file.parent, materia)
            todas_lessons.extend(lessons)

export_to_csv(todas_lessons, output_csv="aulas_todas_materias.csv")


Processando matéria: matematica
Lendo arquivo: 01.04 a 14.04.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 01.08 a 14.08.2024_ Planejamento de Matemática_ 1A.pdf
Lendo arquivo: 04.03 a 17.03.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 05.02 a 18.02.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 05.12 a 17.12.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 07.11 a 20.11.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 10.06 a 23.06.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 10.10 a 23.10.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 12.09 a 25.09.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 13.05 a 26.05.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 15.04  a 28.04.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 15.08  a 28.08.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 18.03 a 31.03.2024_ 1A_ Planejamento de Matemática.pdf
Lendo arquivo: 19.02 a 03.03.2024_ 1A_ Planejamento de 